In [2]:
import numpy as np
import random as r
from engine import Tensor

In [10]:
a = Tensor([[1.,2,3],[4,5,6]])
b = Tensor([2.,1,-1])
c = Tensor([2.,1])
d = Tensor([[1.,1,1],[-1,-1,-1]])
print(a / b.unsqueeze(0))
print(a / c.unsqueeze(1))
print(a / d)

Tensor:
([[ 0.5  2.  -3. ]
 [ 2.   5.  -6. ]])
Grad:
([[0. 0. 0.]
 [0. 0. 0.]])
Tensor:
([[0.5 1.  1.5]
 [4.  5.  6. ]])
Grad:
([[0. 0. 0.]
 [0. 0. 0.]])
Tensor:
([[ 1.  2.  3.]
 [-4. -5. -6.]])
Grad:
([[0. 0. 0.]
 [0. 0. 0.]])
